In [21]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,random_split
import torch.optim as optim
from torchvision.transforms import transforms
from PIL import Image
import scipy
import os
from torch.utils.data import Subset

from utils import OxfordDataset

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
transform_v = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

transform_t = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),        # Flip 50% of images
    transforms.RandomRotation(degrees=15),         # Rotate ±15 degrees
    transforms.ColorJitter(                        # Color augmentation
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.1
    ),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

In [6]:
train_dataset = OxfordDataset("flower_data", transform=transform_v)
validation_dataset = OxfordDataset("flower_data", transform=transform_v)
test_dataset = OxfordDataset("flower_data", transform=transform_v)


In [7]:
train_split = int(0.70 * len(train_dataset))
validation_split = int(0.15 * len(train_dataset))
test_split = len(train_dataset) - train_split - validation_split





In [18]:
indices = torch.randperm(len(train_dataset),generator=torch.Generator().manual_seed(42)).tolist()

train_indices = indices[:train_split]
validation_indices = indices[train_split:train_split+validation_split]
test_indices = indices[train_split+validation_split:]


In [22]:
train_dataset = Subset(train_dataset,train_indices)
val_dataset = Subset(validation_dataset,validation_indices)
test_dataset = Subset(test_dataset,test_indices)

(tensor([[[-1.7069, -1.7240, -1.7240,  ..., -1.8268, -1.7925, -1.7925],
          [-1.7240, -1.7412, -1.7583,  ..., -1.8097, -1.8268, -1.8439],
          [-1.7412, -1.7583, -1.7925,  ..., -1.8097, -1.8610, -1.8953],
          ...,
          [-1.6727, -1.6555, -1.6555,  ...,  1.6667,  2.0605,  2.1462],
          [-1.7240, -1.6727, -1.6213,  ...,  1.0159,  1.6667,  2.0605],
          [-1.7240, -1.6727, -1.6213,  ...,  0.7077,  1.0673,  1.6153]],
 
         [[-1.4755, -1.4930, -1.5455,  ..., -1.1954, -1.2129, -1.2304],
          [-1.4580, -1.4755, -1.5105,  ..., -1.2129, -1.2479, -1.2829],
          [-1.4580, -1.4755, -1.5105,  ..., -1.2479, -1.3354, -1.3880],
          ...,
          [-1.2479, -1.1779, -1.1253,  ...,  1.0805,  1.4832,  1.5707],
          [-1.2479, -1.1954, -1.1429,  ...,  0.4328,  1.0630,  1.4657],
          [-1.2479, -1.1954, -1.1429,  ...,  0.1001,  0.4503,  0.9930]],
 
         [[-1.6476, -1.6650, -1.6824,  ..., -1.7696, -1.7347, -1.6999],
          [-1.6476, -1.6650,

In [20]:
test_indices.__len__()

1229

In [9]:
len(indices)

8189

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_split,batch_size=32, shuffle=False)
test_loader = DataLoader(test_split, batch_size=32, shuffle=False)


In [8]:
for image,label in train_loader:
    print(label)
    print(label[0].item())
    print(image.shape)
    break



tensor([ 53,  59,  50,  50,  59,  92,  87,  87,  81,  81,  65,  74,  87,   2,
         86,  88,  49,  71,  50,  55,  87,  63,  72,  73,  50,  58,  85,  18,
         45, 100,  64,  38], dtype=torch.uint8)
53
torch.Size([32, 3, 224, 224])


In [9]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32,kernel_size=3,padding=1,stride=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,kernel_size=3,stride=2,padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3,stride=1,padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv2d(in_channels=128, out_channels=512,kernel_size=3,stride=1,padding=1)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512*3*3,512)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(512,102)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)  
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x



In [10]:
model = SimpleCNN().to(device)

optimizer = optim.Adam(model.parameters(),lr=0.0005, weight_decay=0.0005)
loss_function = nn.CrossEntropyLoss()


In [11]:
def train_epoch(model, train_loader, optimizer, loss_function):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if (batch_idx + 1) % 10 == 0:  # Changed condition
            avg_loss = running_loss / 10  # Now correct
            acc = 100. * correct / total
            print(f' [{(batch_idx + 1) * len(data)}/{len(train_loader.dataset)}]'
                  f' Loss: {avg_loss:.3f} | Accuracy: {acc:.1f} %')
            running_loss = 0.0  # Only reset loss, keep accuracy cumulative

In [12]:
def validation(model,validation_loader,device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx, (data, targets) in enumerate(validation_loader):
            data, targets = data.to(device), targets.to(device)
            output = model(data)
            _,predicted = output.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return 100. * correct/total


In [15]:
num_epochs = 10

for idx in range(num_epochs):
    print(f"Epoch : {idx}")
    train_epoch(model,train_loader,optimizer,loss_function)
    validation_acc = validation(model, validation_loader, device)
    print(f"Validation Accuracy : {validation_acc}")



Epoch : 0
 [320/5732] Loss: 0.853 | Accuracy: 74.7 %
 [640/5732] Loss: 0.888 | Accuracy: 74.1 %
 [960/5732] Loss: 0.923 | Accuracy: 73.1 %
 [1280/5732] Loss: 0.889 | Accuracy: 73.4 %
 [1600/5732] Loss: 0.885 | Accuracy: 73.1 %
 [1920/5732] Loss: 0.957 | Accuracy: 72.7 %
 [2240/5732] Loss: 0.824 | Accuracy: 73.4 %
 [2560/5732] Loss: 0.819 | Accuracy: 73.5 %
 [2880/5732] Loss: 0.861 | Accuracy: 73.6 %
 [3200/5732] Loss: 0.873 | Accuracy: 73.6 %
 [3520/5732] Loss: 0.897 | Accuracy: 73.7 %
 [3840/5732] Loss: 0.952 | Accuracy: 73.6 %
 [4160/5732] Loss: 0.921 | Accuracy: 73.3 %
 [4480/5732] Loss: 0.853 | Accuracy: 73.4 %
 [4800/5732] Loss: 0.821 | Accuracy: 73.6 %
 [5120/5732] Loss: 0.823 | Accuracy: 73.6 %
 [5440/5732] Loss: 0.931 | Accuracy: 73.4 %
 [720/5732] Loss: 0.839 | Accuracy: 73.5 %
Validation Accuracy : 62.05211726384365
Epoch : 1
 [320/5732] Loss: 0.839 | Accuracy: 74.1 %
 [640/5732] Loss: 0.753 | Accuracy: 75.8 %
 [960/5732] Loss: 0.728 | Accuracy: 75.9 %
 [1280/5732] Loss: 0.76

KeyboardInterrupt: 